In [1]:
import pandas as pd
import numpy as np
from io import StringIO

<H4>
Files imported: /data/PowerSchoolSFTP/Students_export.csv, /data/PowerSchoolSFTP/ContactsPowerQuery.csv <br>
File exported: StudentContactInfo.csv <br>
<br>
Product is the necessary Student & Parent/Guardian contact info the eIEPPROStudentDemographics.csv requires

The plan: <br>
 * get the contact fields (using custody flag) per student <br>
 * put in order of student id (new index) <br> 
 * pick first mother, father, guardian contacts <br> 
 * flush out required fields and transpose <br>
 * join them with the "Active Child Find" students from the other file <br>
<br>
Key = Student Number, STUDENTS.Student_Number from Students_exp <br>

Spec for - concentrating on Parents in this file 

| Data Value | e-IEPPRO Column Name | Details | 
| --- |--- |--- |
| Parent #1 Name              | Parent1Name         | --- |
| Parent #1 Address-1         | Parent1Address1     | Street Address |
| Parent #1 Address-2         | Parent1Address2     | City, State and Zip Code |  
| Parent #1 Home Phone        | Parent1HomePhone    | --- |
| Parent #1 Work Phone        | Parent1WorkPhone    | --- |
| Parent #1 Cell Phone        | Parent1CellPhone    | --- |
| Parent #1 Relationship      | Parent1Relationship | --- |
| Parent #1 Email             | Parent1Email        | --- |
| Parent #2 Name              | Parent2Name         | --- |
| Parent #2 Address-1         | Parent2Address1     | Street Address |
| Parent #2 Address-2         | Parent2Address2     | City, State and Zip Code |  
| Parent #2 Home Phone        | Parent2HomePhone    | --- |
| Parent #2 Work Phone        | Parent2WorkPhone    | --- |
| Parent #2 Cell Phone        | Parent2CellPhone    | --- |
| Parent #2 Relationship      | Parent2Relationship | --- |
| Parent #2 Email             | Parent2Email        | --- |

| --- |--- |--- |
| --- |--- |--- |

In [9]:
StudentAddress  = pd.read_csv("Students_export.csv", encoding='latin-1') 
StudentContacts = pd.read_csv("ContactsPowerQuery.csv", encoding='latin-1')

<H5> Find the subset of student contacts that are 504 and/or Idea

In [13]:
# Pull the intended population
Students_504 = StudentAddress.loc[StudentAddress['ACTIVITIES.childfind504'] == 1] 
Students_and_idea = StudentAddress.loc[StudentAddress['ACTIVITIES.childfindidea'] == 1]
df_504_and_Idea = pd.concat([Students_504, Students_and_idea], ignore_index=True).drop_duplicates()

In [14]:
df_504_and_Idea

,STUDENTS.SchoolID,STUDENTS.ID,STUDENTS.Student_Number,STUDENTS.Last_Name,STUDENTS.First_Name,STUDENTS.DOB,STUDENTS.Mother,STUDENTS.Father,STUDENTS.Grade_Level,STUDENTS.Home_Room,...,U_DEF_EXT_STUDENTS.sibling5hr,ELL Alert,ACTIVITIES.indicator1,ACTIVITIES.indicator2,ACTIVITIES.childfind504,ACTIVITIES.childfindidea,504 Alert,S_AZ_STU_X.firstacquiredlanguage,S_AZ_STU_X.homelanguagecode,S_AZ_STU_X.spokenlanguage
0,450,55,90001285,Jensen,Cash,2008-12-17,"Foshee, Heather","Jensen, Blake",8,"Ocasla-Johnson, Samantha",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
1,100,1500,20081142,Napier,Timothy,2009-03-08,"Napier, Rebecca","Napier, Timothy",8,"Brownfield, Cambria",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
2,350,2621,70001095,Dohaniuk,Jordan,2009-06-01,"Dohaniuk, Beth","Dohaniuk, Jeffrey",8,"Grijalva, Alex",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
3,350,2623,70001183,Roberts,Katelyn,2009-01-09,"Roberts, Kimberly","Roberts, Jeffery",8,"Grijalva, Alex",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
4,350,2655,70001083,Green,Cody,2008-12-18,"Green, Mindy","Green, Jason",8,"Spencer, Megan",...,.,NaN,NaN,NaN,1.0,NaN,Y,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,500,224977,76386,Farooqi,Christopher,2010-11-22,"Farooqi, Bella","Farooqi, Rizwan",6,Bradbury,...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0
2624,1150,224983,76392,Davy,Adalynn,2012-12-29,"Davy, Kimberly",NaN,4,"Thomas, Debra",...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0
2625,1150,224987,76396,White,Jonathan,2009-03-06,"White, Sandra",NaN,7,"Richard, Jill",...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0
2626,1000,225011,76408,Hudson,Deshawn,2012-11-09,"Hudson, Shaqueila","Sherrill, Demylo",4,"Papamatheakis, Athena",...,.,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0.0


In [25]:
# Use the STUDENTS.ID field to pull the necessary contacts
df_StudentsWeNeed = df_504_and_Idea.filter(['STUDENTS.ID', 'STUDENTS.Student_Number']).drop_duplicates()



In [26]:
df_StudentsWeNeed.head()

,STUDENTS.ID,STUDENTS.Student_Number
0,55,90001285
1,1500,20081142
2,2621,70001095
3,2623,70001183
4,2655,70001083


In [31]:
# Contacts for _ALL_ students in PS
StudentContacts

,Contact ID,Contact First Name,Contact Last Name,Relationship,End Date,Custody Flag,School Pick Up,Emergency Contact,Email Address,Phone Number,Student Number,Student LastFirst
0,402001,Brandi,Adams Bressler,Mother,NaN,1,1,1,brandimoriah@msn.com,602-370-8931,80001037,"Bressler, Charlotte Moriah"
1,402001,Brandi,Adams Bressler,Friend,NaN,0,0,1,brandimoriah@msn.com,602-370-8931,80001135,"Woodruff, Grace Marie"
2,402006,Phaedra,Toth-Albrecht,Mother,NaN,1,0,0,phaedrat66@gmail.com,520-252-1612,10035,"Toth-Albrecht, Aleah"
3,402008,Kellie,Cook,Mother,NaN,1,0,0,Cookkw2002@yahoo.com,520-252-7361,10801,"Cook, Victoria Lyn"
4,402009,Brittany,Almaraz,Mother,NaN,1,0,0,theonlybrittany@yahoo.com,520-483-4898,10797,"Almaraz, Aiden Andrew"
...,...,...,...,...,...,...,...,...,...,...,...,...
100359,31812972,Christopher,Ruiz,Stepfather,NaN,0,1,1,NaN,623-330-2765,76483,"Love, Aaliyah Sanai"
100360,31812973,Sandra,Kachelski,Grandmother,NaN,0,0,1,NaN,760-646-0335,21774,"Ware, Ethan Michael"
100361,31812973,Sandra,Kachelski,Grandmother,NaN,0,0,1,NaN,760-646-0335,21775,"Ware, Mason Christopher"
100362,31812973,Sandra,Kachelski,Grandmother,NaN,0,0,1,NaN,760-646-0335,45762,"Ware, Caleb David"


In [ ]:
# Running this for StudentContacts, only
# Fields e-IEPPRO wants are Parent# Name, Address, Home Phone, Work Phone, Cell Phone, Relationship, Email
# Fields the ContactsQuery offers are: Name, "Phone Number", Relationship, Email Address

def BuildParentContactsV3( df_ContactsPQ, df_ESS_Students ):
    global df_PContacts
    df_PContacts = pd.DataFrame()
    """ Create a counter for each student ID """
    for index, row in df_ContactsPQ.iterrows():
        parent_id = row['Contact ID']
        student_ID = row['Student Number']
        relationship = row['Relationship'] 
        ParentName = row['Contact First Name'] + ' ' + row['Contact Last Name']
        ParentPhone = row['Phone Number']
        ParentEmail = row['Email Address']
        
        df_PContacts.at[index, 'student_ID'] = student_ID
        
        match relationship:
            case 'Mother':
                df_PContacts.at[index, 'Mother'] = parent_id
                df_PContacts.at[index, 'Parent1Name'] = ParentName
                df_PContacts.at[index, 'Parent1CellPhone'] = ParentPhone
                df_PContacts.at[index, 'Parent1Email'] = ParentEmail
            case 'Father':
                df_PContacts.at[index, 'Father'] = parent_id
                df_PContacts.at[index, 'Parent2Name'] = ParentName
                df_PContacts.at[index, 'Parent2CellPhone'] = ParentPhone
                df_PContacts.at[index, 'Parent2Email'] = ParentEmail
            case 'Guardian':
                df_PContacts.at[index, 'Guardian'] = parent_id  
                df_PContacts.at[index, 'GuardianName'] = ParentName
                df_PContacts.at[index, 'GuardianCellPhone'] = ParentPhone
                df_PContacts.at[index, 'GuardianEmail'] = ParentEmail
    return df_PContacts

In [12]:
# Fields e-IEPPRO wants are Parent# Name, Address, Home Phone, Work Phone, Cell Phone, Relationship, Email
# Fields the ContactsQuery offers are: Name, "Phone Number", Relationship, Email Address

def BuildParentContactsV2( df_ContactsPQ ):
    global df_PContacts
    df_PContacts = pd.DataFrame()
    """ Create a counter for each student ID """
    for index, row in df_ContactsPQ.iterrows():
        parent_id = row['Contact ID']
        student_ID = row['Student Number']
        relationship = row['Relationship'] 
        ParentName = row['Contact First Name'] + ' ' + row['Contact Last Name']
        ParentPhone = row['Phone Number']
        ParentEmail = row['Email Address']
        
        df_PContacts.at[index, 'student_ID'] = student_ID
        
        match relationship:
            case 'Mother':
                df_PContacts.at[index, 'Mother'] = parent_id
                df_PContacts.at[index, 'Parent1Name'] = ParentName
                df_PContacts.at[index, 'Parent1CellPhone'] = ParentPhone
                df_PContacts.at[index, 'Parent1Email'] = ParentEmail
            case 'Father':
                df_PContacts.at[index, 'Father'] = parent_id
                df_PContacts.at[index, 'Parent2Name'] = ParentName
                df_PContacts.at[index, 'Parent2CellPhone'] = ParentPhone
                df_PContacts.at[index, 'Parent2Email'] = ParentEmail
            case 'Guardian':
                df_PContacts.at[index, 'Guardian'] = parent_id  
                df_PContacts.at[index, 'GuardianName'] = ParentName
                df_PContacts.at[index, 'GuardianCellPhone'] = ParentPhone
                df_PContacts.at[index, 'GuardianEmail'] = ParentEmail
    return df_PContacts

In [32]:
# merged.rename(columns={'STUDENTS.SchoolID': 'school_sis_id', 'STUDENTS.Grade_Level': 'grade', 'Email Address': 'email'}, inplace=True)
df_StudentsWeNeed.rename(columns={'STUDENTS.Student_Number' : 'student_number'}, inplace=True)
StudentContacts.rename(columns={'Student Number' : 'student_number'}, inplace=True)

In [33]:
StudentContacts

,Contact ID,Contact First Name,Contact Last Name,Relationship,End Date,Custody Flag,School Pick Up,Emergency Contact,Email Address,Phone Number,student_number,Student LastFirst
0,402001,Brandi,Adams Bressler,Mother,NaN,1,1,1,brandimoriah@msn.com,602-370-8931,80001037,"Bressler, Charlotte Moriah"
1,402001,Brandi,Adams Bressler,Friend,NaN,0,0,1,brandimoriah@msn.com,602-370-8931,80001135,"Woodruff, Grace Marie"
2,402006,Phaedra,Toth-Albrecht,Mother,NaN,1,0,0,phaedrat66@gmail.com,520-252-1612,10035,"Toth-Albrecht, Aleah"
3,402008,Kellie,Cook,Mother,NaN,1,0,0,Cookkw2002@yahoo.com,520-252-7361,10801,"Cook, Victoria Lyn"
4,402009,Brittany,Almaraz,Mother,NaN,1,0,0,theonlybrittany@yahoo.com,520-483-4898,10797,"Almaraz, Aiden Andrew"
...,...,...,...,...,...,...,...,...,...,...,...,...
100359,31812972,Christopher,Ruiz,Stepfather,NaN,0,1,1,NaN,623-330-2765,76483,"Love, Aaliyah Sanai"
100360,31812973,Sandra,Kachelski,Grandmother,NaN,0,0,1,NaN,760-646-0335,21774,"Ware, Ethan Michael"
100361,31812973,Sandra,Kachelski,Grandmother,NaN,0,0,1,NaN,760-646-0335,21775,"Ware, Mason Christopher"
100362,31812973,Sandra,Kachelski,Grandmother,NaN,0,0,1,NaN,760-646-0335,45762,"Ware, Caleb David"


In [30]:
df_StudentsWeNeed

,STUDENTS.ID,student_number
0,55,90001285
1,1500,20081142
2,2621,70001095
3,2623,70001183
4,2655,70001083
...,...,...
2623,224977,76386
2624,224983,76392
2625,224987,76396
2626,225011,76408


In [34]:
# Perhaps we do a merge first, and then create the file
# merged = pd.merge(EmailContacts, SG_copy, left_on= "Student Number", right_on= 'STUDENTS.Student_Number', how= "inner")
df_ESS_Students = pd.merge(df_StudentsWeNeed, StudentContacts, left_on = 'student_number', right_on = 'student_number', how = 'inner')

In [35]:
df_ESS_Students

,STUDENTS.ID,student_number,Contact ID,Contact First Name,Contact Last Name,Relationship,End Date,Custody Flag,School Pick Up,Emergency Contact,Email Address,Phone Number,Student LastFirst
0,55,90001285,406614,Heather,Foshee,Mother,NaN,1,0,0,heathermfoshee@gmail.com,480-438-8430,"Jensen, Cash Wyatt"
1,55,90001285,410374,Blake,Jensen,Father,NaN,1,0,0,perfectiongraniteco@yahoo.com,480-233-2754,"Jensen, Cash Wyatt"
2,55,90001285,415615,Brenda,Jensen,Other,NaN,0,0,1,NaN,480-459-0354,"Jensen, Cash Wyatt"
3,55,90001285,415616,Kali,Mattson,Stepmother,NaN,0,0,1,NaN,480-815-0098,"Jensen, Cash Wyatt"
4,55,90001285,415617,Kelly,Jensen,Other,NaN,0,0,1,NaN,480-459-0353,"Jensen, Cash Wyatt"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12745,225011,76408,31811706,Demylo,Sherrill,Father,NaN,1,1,1,NaN,NaN,"Hudson, Deshawn James"
12746,225011,76408,31812292,Cory,Little,Grandfather,NaN,0,1,0,NaN,NaN,"Hudson, Deshawn James"
12747,225015,76412,31811433,Jonathan,Williams,Father,NaN,1,0,0,Jonathan.Williams.479@gmail.com,512-781-4968,"Williams, Cayden"
12748,225015,76412,31811434,Kamille,Williams,Mother,NaN,1,0,0,Kamillewilliams4@gmail.com,512-781-4967,"Williams, Cayden"


In [38]:
# Fields e-IEPPRO wants are Parent# Name, Address, Home Phone, Work Phone, Cell Phone, Relationship, Email
# Fields the ContactsQuery offers are: Name, "Phone Number", Relationship, Email Address

def BuildParentContactsV2( df_ContactsPQ ):
    global df_PContacts
    df_PContacts = pd.DataFrame()
    """ Create a counter for each student ID """
    for index, row in df_ContactsPQ.iterrows():
        parent_id = row['Contact ID']
        student_ID = row['student_number']
        relationship = row['Relationship'] 
        ParentName = row['Contact First Name'] + ' ' + row['Contact Last Name']
        ParentPhone = row['Phone Number']
        ParentEmail = row['Email Address']
        
        df_PContacts.at[index, 'student_ID'] = student_ID
        
        match relationship:
            case 'Mother':
                df_PContacts.at[index, 'Mother'] = parent_id
                df_PContacts.at[index, 'Parent1Name'] = ParentName
                df_PContacts.at[index, 'Parent1CellPhone'] = ParentPhone
                df_PContacts.at[index, 'Parent1Email'] = ParentEmail
            case 'Father':
                df_PContacts.at[index, 'Father'] = parent_id
                df_PContacts.at[index, 'Parent2Name'] = ParentName
                df_PContacts.at[index, 'Parent2CellPhone'] = ParentPhone
                df_PContacts.at[index, 'Parent2Email'] = ParentEmail
            case 'Guardian':
                df_PContacts.at[index, 'Guardian'] = parent_id  
                df_PContacts.at[index, 'GuardianName'] = ParentName
                df_PContacts.at[index, 'GuardianCellPhone'] = ParentPhone
                df_PContacts.at[index, 'GuardianEmail'] = ParentEmail
    return df_PContacts

In [39]:
# Runs in ~ 9 minutes 27 seconds minutes on my local computer
# Takes all Contacts and creates ~100,364 rows
# Efficiency measure: pull 504Idea community first?
TestContacts = BuildParentContactsV2(df_ESS_Students)

In [8]:
TestContacts

,student_ID,Mother,Parent1Name,Parent1CellPhone,Parent1Email,Father,Parent2Name,Parent2CellPhone,Parent2Email,Guardian,GuardianName,GuardianCellPhone,GuardianEmail
0,80001037.0,402001.0,Brandi Adams Bressler,602-370-8931,brandimoriah@msn.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,80001135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10035.0,402006.0,Phaedra Toth-Albrecht,520-252-1612,phaedrat66@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10801.0,402008.0,Kellie Cook,520-252-7361,Cookkw2002@yahoo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10797.0,402009.0,Brittany Almaraz,520-483-4898,theonlybrittany@yahoo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100359,76483.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100360,21774.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100361,21775.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100362,45762.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
TestContacts.to_csv('TestContacts.csv', encoding='utf-8')

In [41]:
df_StudentContactsToWrite = TestContacts.groupby('student_ID').agg('first')

In [42]:
#  Down to 21,522 rows now (approx our AZ student body, we needed this data, regardless)
df_StudentContactsToWrite

,Mother,Parent1Name,Parent1CellPhone,Parent1Email,Father,Parent2Name,Parent2CellPhone,Parent2Email,Guardian,GuardianName,GuardianCellPhone,GuardianEmail
student_ID,,,,,,,,,,,,
10035.0,402006.0,Phaedra Toth-Albrecht,520-252-1612,phaedrat66@gmail.com,NaN,None,None,None,NaN,None,None,None
10146.0,402356.0,Leslie Walther,480-330-6086,lxgrimm@yahoo.com,505169.0,Keith Walther,480-330-6086,None,NaN,None,None,None
10180.0,406772.0,Ashleigh Morris,480-282-1652,FREERIDEBIKECO@GMAIL.COM,505447.0,Brian Morris,480-695-4095,None,NaN,None,None,None
10230.0,405511.0,Kimberly Moise,480-888-6180,kmoise0124@gmail.com,NaN,None,None,None,NaN,None,None,None
10433.0,506785.0,Carmen Torres,623-800-9739,gian.torres2009@gmail.com,506787.0,Jose Torres,623-293-7534,None,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
80001323.0,406277.0,Tawnia Randolph,480-246-5184,tawniam0526@gmail.com,526696.0,Chris Randolph,480-516-2481,None,NaN,None,None,None
90001271.0,405384.0,Christina Mirand,602-722-8660,vemmaninjas@gmail.com,415707.0,Nate Mirand,602-722-7826,Nate.mirand@gmail.com,NaN,None,None,None
90001285.0,406614.0,Heather Foshee,480-438-8430,heathermfoshee@gmail.com,410374.0,Blake Jensen,480-233-2754,perfectiongraniteco@yahoo.com,NaN,None,None,None


In [43]:
df_StudentContactsToWrite.to_csv('StudentContactsBeta.csv', encoding='utf-8')

<H2> The join below results in 4,207 students and parent contacts matched

In [ ]:
df_504_ID_Contacts = df_StudentInfo_504_idea.join(df_CustConStuNum, on='Student Number').drop_duplicates()

In [ ]:
df_504_ID_Contacts

In [ ]:
df_504_ID_Contacts.info()

In [ ]:
df_504_ID_Contacts

In [ ]:
# Results in 4,207 Contacts - some students have one, some three, mostly two
df_504_ID_Contacts.to_csv('eIEPPRO_Contacts.csv', encoding='utf-8')

<H1> Break here

In [ ]:
# Next up, answer questions:
# Who is not in 504_idea that is in this file?
# who is in 504_idea that is not in this file?
# Let's start with reorganizing the file for student name visibility

In [ ]:
df_StudentInfo_504_idea.info()

In [ ]:
list_504_and_idea = df_StudentInfo_504_idea['Student Number'].reset_index(drop=True)

In [ ]:
list_504_and_idea

In [ ]:
df_CustodyContacts['ParentName'] =  df_CustodyContacts['Contact Last Name'].map(str) + ", " + df_CustodyContacts['Contact First Name'].map(str)

Need to determine "Two Best" contacts

| Data Value | e-IEPPRO Column Name | Details | 
| --- |--- |--- |
| Student Address #1          | StudentAddress1     | Powerschool STUDENTS.Street ~ Street Address |
| Student Address #2          | StudentAddress2     | City, State and Zip Code |
| Student Physical Address    | StudentPhyAddress   | --- |  
| Student Phone #             | StudentPhone        | --- |
| Student E-Mail              | StudentEmail        | --- |
| Parent #1 Name              | Parent1Name         | --- |
| Parent #1 Address-1         | Parent1Address1     | Street Address |
| Parent #1 Address-2         | Parent1Address2     | City, State and Zip Code |  
| Parent #1 Home Phone        | Parent1HomePhone    | --- |
| Parent #1 Work Phone        | Parent1WorkPhone    | --- |
| Parent #1 Cell Phone        | Parent1CellPhone    | --- |
| Parent #1 Relationship      | Parent1Relationship | --- |
| Parent #1 Email             | Parent1Email        | --- |
| Parent #2 Name              | Parent2Name         | --- |
| Parent #2 Address-1         | Parent2Address1     | Street Address |
| Parent #2 Address-2         | Parent2Address2     | City, State and Zip Code |  
| Parent #2 Home Phone        | Parent2HomePhone    | --- |
| Parent #2 Work Phone        | Parent2WorkPhone    | --- |
| Parent #2 Cell Phone        | Parent2CellPhone    | --- |
| Parent #2 Relationship      | Parent2Relationship | --- |
| Parent #2 Email             | Parent2Email        | --- |

| --- |--- |--- |

In [ ]:
df_ContactsPivot = df_CustodyContacts.copy()

In [ ]:
df_ContactsPivot.info()

In [ ]:
df_ContactsbyStudentNum = df_ContactsPivot[['Student Number', 'Student LastFirst', 'Contact ID', 'ParentName', 'Relationship', 
                                           'Email Address', 'Phone Number']].drop_duplicates().reset_index(drop=True)

In [ ]:
df_ContactsbyStudentNum

In [ ]:
df_ContactsbyStudentNum.to_csv('df_ContactsbyStudentNum.csv', encoding='utf-8')

In [ ]:
df_ContactsbyStudentNum['Student Number'].value_counts()

In [ ]:
df_ContactsPivot.pivot(index='Student Number', columns= ['Contact First Name', 'Contact Last Name'], values = ['Contact First Name', 'Contact Last Name'])